In [ ]:
! pip install wandb

In [3]:
import time
import tensorflow as tf
from tensorflow.keras.layers import Flatten,Dense,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import wandb
from wandb.keras import WandbCallback

import numpy as np
import cv2
import matplotlib.pyplot as plt

In [4]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
wandb.init(project="17 Flowers", entity="gharabadiyan")

wandb: Currently logged in as: gharabadiyan. Use `wandb login --relogin` to force relogin


## Dataset

In [6]:
train_dataset_path = '/content/drive/MyDrive/Flowers/Train'
width = height = 224
batch_size = 32

idg = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    brightness_range = (0.8,1.2),
    zoom_range = 0.1,
    shear_range = 0.3,
    rotation_range = 10,
    validation_split = 0.2
)

train_data = idg.flow_from_directory(
    train_dataset_path,
    target_size = (width,height),
    class_mode = "categorical",
    batch_size = batch_size,
    subset = "training"

)
val_data = idg.flow_from_directory(
    train_dataset_path,
    target_size = (width,height),
    class_mode = "categorical",
    batch_size = batch_size,
    subset = "validation"

)

Found 884 images belonging to 17 classes.
Found 204 images belonging to 17 classes.


## Model

In [7]:
base_model = tf.keras.applications.MobileNetV2(
    include_top=False,
    weights = 'imagenet',
    input_shape=(width,height,3),
    pooling = 'avg',
)

model = tf.keras.models.Sequential([
        base_model,
        Dropout(0.3), 
        # Flatten(),
        # Dense(1024,activation='relu'),
        # Dense(256,activation='relu'),
        Dense(17,activation='softmax')

])

9420800/9406464 [==============================] - 0s 0us/step


In [8]:
for layer in base_model.layers[:-2]:  #freeze layer
  layer.trainable = False

In [9]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

In [10]:
model.fit(train_data,validation_data=val_data,epochs=10,callbacks=[WandbCallback()])

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`


Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`


Epoch 1/10
28/28 [==============================] - 528s 19s/step - loss: 2.4549 - accuracy: 0.2771 - val_loss: 1.4177 - val_accuracy: 0.6078 - _timestamp: 1658382140.0000 - _runtime: 597.0000
Epoch 2/10
28/28 [==============================] - 43s 2s/step - loss: 1.1884 - accuracy: 0.6538 - val_loss: 0.8182 - val_accuracy: 0.8137 - _timestamp: 1658382220.0000 - _runtime: 677.0000
Epoch 3/10
28/28 [==============================] - 42s 1s/step - loss: 0.7399 - accuracy: 0.8133 - val_loss: 0.6369 - val_accuracy: 0.8284 - _timestamp: 1658382302.0000 - _runtime: 759.0000
Epoch 4/10
28/28 [==============================] - 42s 1s/step - loss: 0.5687 - accuracy: 0.8518 - val_loss: 0.5065 - val_accuracy: 0.8676 - _timestamp: 1658382343.0000 - _runtime: 800.0000
Epoch 5/10
28/28 [==============================] - 42s 2s/step - loss: 0.4389 - accuracy: 0.8914 - val_loss: 0.4352 - val_accuracy: 0.8824 - _timestamp: 1658382386.0000 - _runtime: 843.0000
Epoch 6/10
28/28 [=========================

In [11]:
test_dataset_path = '/content/drive/MyDrive/Flowers/Test'
width=height = 224
batch_size = 32
idg = ImageDataGenerator(
    rescale = 1./255,
)


test_data = idg.flow_from_directory(
    test_dataset_path,
    target_size = (width,height),
    class_mode = "categorical",
    batch_size = batch_size,
)

Found 272 images belonging to 17 classes.


In [12]:
model.evaluate(test_data)

9/9 [==============================] - 110s 14s/step - loss: 0.5871 - accuracy: 0.8162


[0.5871051549911499, 0.8161764740943909]

In [14]:
img = cv2.imread('/content/images (2).jpg')
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
img = cv2.resize(img,(width,height))
img = img / 255.0

img = img.reshape(1,width,height,3)

start_time = time.time()
result = model.predict(img)
end_time = time.time()

print(np.argmax(result))
print(end_time - start_time)


9
0.7624733448028564
